In [ ]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - INAR                                ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#                                                        ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        29/10/2025  -  23:00:15           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    04/11/2025  -  12:45:31           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#====================================================================================================#

# Iteración 3 - Transfer Learning VGG16 (Modo Categorical)

Primera prueba con Transfer Learning usando VGG16 pre-entrenado. Aquí todavía tengo el bug de usar categorical en vez de binary.

**Arquitectura**: VGG16 (congelado) + Dense(256) + Dropout(0.5) + Dense(2, softmax)

**Kaggle Score**:  0.54664 (menor que iteración 1 por el bug de categorical)

In [ ]:
# Imports básicos
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import data as tf_data
import keras

seed = 42
keras.utils.set_random_seed(seed)

DATASET_NAME = "u-tad-dogs-vs-cats-2025"
TRAIN_PATH = f"/kaggle/input/{DATASET_NAME}/train/train"
TEST_PATH = f"/kaggle/input/{DATASET_NAME}/test/test"
SUPP_PATH = f"/kaggle/input/{DATASET_NAME}/supplementary_data/supplementary_data"

print("Versión de Keras:", keras.__version__)

In [ ]:
# Cargo datos con VGG16 size (224x224) pero todavía en modo categorical (bug)
image_size = (224, 224)
batch_size = 125

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    validation_split=0.2,
    subset="both",
    seed=seed,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="categorical",  # BUG: debería ser binary
)

print(f"Training batches: {len(train_ds)}")
print(f"Validation batches: {len(val_ds)}")

In [ ]:
# Data augmentation para Transfer Learning con VGG16
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
], name="data_augmentation")

print("Data Augmentation configurado")

In [ ]:
# Cargo VGG16 pre-entrenado y congelo todas las capas
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.applications import VGG16

input_shape = image_size + (3,)

base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape,
    pooling='avg'
)

base_model.trainable = False

# Modelo completo con salida softmax(2) por el bug
model = Sequential([
    keras.Input(shape=input_shape),
    data_augmentation,
    base_model,
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # BUG: debería ser Dense(1, sigmoid)
], name='VGG16_Transfer_Learning')

print(f"Capas VGG16 congeladas: {len(base_model.layers)}")
model.summary()

In [ ]:
# Compilo y entreno
%%time

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',  # BUG: debería ser binary_crossentropy
    metrics=['accuracy']
)

epochs = 15

print(f"Épocas: {epochs}")
print(f"Optimizer: Adam (lr=0.0001)")
print("-" * 60)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    verbose=1
)

print("-" * 60)
print(f"Val Accuracy final: {history.history['val_accuracy'][-1]:.4f}")

In [ ]:
# Pinto las curvas
logs = pd.DataFrame(history.history)

plt.figure(figsize=(14, 4))

plt.subplot(1, 2, 1)
plt.plot(logs.loc[1:, "loss"], lw=2, label='Pérdida en entrenamiento')
plt.plot(logs.loc[1:, "val_loss"], lw=2, label='Pérdida en validación')
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(logs.loc[1:, "accuracy"], lw=2, label='Precisión en entrenamiento')
plt.plot(logs.loc[1:, "val_accuracy"], lw=2, label='Precisión en validación')
plt.xlabel("Época")
plt.ylabel("Precisión")
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPrecisión final en entrenamiento: {logs['accuracy'].iloc[-1]:.4f}")
print(f"Precisión final en validación: {logs['val_accuracy'].iloc[-1]:.4f}")

In [ ]:
# Guardo el modelo
model.save("model.keras")
print("Modelo guardado como 'model.keras'")

In [ ]:
# Evalúo con supplementary
supplementary_ds = keras.utils.image_dataset_from_directory(
    SUPP_PATH,
    image_size=image_size,
    batch_size=batch_size,
    labels="inferred",
    label_mode="categorical",
)

print("Evaluando con datos suplementarios...")
results = model.evaluate(supplementary_ds, return_dict=True, verbose=1)

print(f"\nSupplementary Accuracy: {results['accuracy']:.4f}")

In [ ]:
# Genero predicciones usando argmax (categorical)
%%time

predictions_dict = {}

print(f"Generando predicciones para {len(os.listdir(TEST_PATH))} imágenes...")

for img in os.listdir(TEST_PATH):
    img_path = os.path.join(TEST_PATH, img)
    file_name = img_path.split('/')[-1]
    file_no_extension = file_name.split('.')[0]
    
    img_loaded = keras.utils.load_img(img_path, target_size=image_size)
    img_array = keras.utils.img_to_array(img_loaded)
    img_array = keras.ops.expand_dims(img_array, 0)
    
    prediction = model.predict(img_array, verbose=0)
    label = int(keras.ops.argmax(prediction[0]).numpy())
    
    predictions_dict[int(file_no_extension)] = label

print(f"Predicciones completadas: {len(predictions_dict)}")

In [ ]:
# Creo submission (mismo score que iteración 1 por el bug)
submission = pd.DataFrame(predictions_dict.items(), columns=["id", "label"])
submission = submission.sort_values(by='id', ascending=True)
submission.to_csv('submission.csv', index=False)

print("Archivo de submission creado")
print("\nDistribución de predicciones:")
print(submission["label"].value_counts())
print(f"\nTotal: {len(submission)} imágenes")